In [1]:
%load_ext autoreload
from ddls.devices.processors.gpus.A100 import A100
from ddls.plotting.plotting import plot_computation_graph
from ddls.environments.ramp_job_placement_shaping.ramp_job_placement_shaping_environment import RampJobPlacementShapingEnvironment
from ddls.demands.jobs.job import Job
from ddls.distributions.uniform import Uniform
from ddls.utils import seed_stochastic_modules_globally

from ddls.ml_models.policies import GNNPolicy
from ddls.plotting.plotting import plot_line

from ddls.loops.rllib_epoch_loop import RLlibEpochLoop
from ddls.launchers.launcher import Launcher

import ray
from ray.tune.registry import register_env
from ray.tune.logger import pretty_print
ray.shutdown()
ray.init()

from ray.rllib.models import ModelCatalog
from ray.rllib.agents import ppo

import glob
import numpy as np
from collections import defaultdict

2022-06-23 12:56:17,686	INFO services.py:1338 -- View the Ray dashboard at http://127.0.0.1:8269
2022-06-23 12:56:24,346	INFO services.py:1338 -- View the Ray dashboard at http://127.0.0.1:8269


In [2]:
%autoreload
register_env('ramp_job_placement_shaping_environment', lambda env_config: RampJobPlacementShapingEnvironment(**env_config))

In [3]:
%autoreload
ModelCatalog.register_custom_model('my_model', GNNPolicy)

In [4]:
%autoreload

node_config = {'type_1':
                  {
                      'num_nodes': 16, # 8 16
                      'workers_config': 
                          [
                              {
                               'num_workers': 1, # NEED 1 WORKER PER SERVER FOR RAMP
                               'worker': A100
                              }
                          ]
                  }
              }

topology_config = {'type':
                      'ramp',
                   'kwargs':
                      {
                          'num_communication_groups': 2,
                          'num_racks_per_communication_group': 2,
                          'num_servers_per_rack': 4, # 2 4
                          'num_channels': 2
                      }
                  }

jobs_config = {'path_to_files': '/scratch/datasets/ddls/jobs/pipedream_graphs/image_classification/profiles/alexnet/',
               'job_interarrival_time_dist': Uniform(min_val=1, max_val=1000),
               'max_files': 20,
               'job_sampling_mode': 'remove',
               # 'job_sampling_mode': 'remove_and_replace',
               }

In [5]:
env_config = {'node_config': node_config,
              'topology_config': topology_config,
              'jobs_config': jobs_config,
              # 'max_simulation_run_time': 1e4,
              'max_simulation_run_time': float('inf'),
              'job_queue_capacity': 100,
              'reward_function': 'lookahead_job_completion_time',
              'pad_obs_kwargs': {'max_nodes': 200}
             }

model_config = {
        'in_features_node':5,
        'in_features_edge':1,
        'out_features_msg':8,
        'out_features_hidden':16,
        'out_features':4,
        'in_features_graph':34, # CHANGE 130
        'out_features_graph':4,
        'num_layers':1,
        'aggregator_type':'mean',
        'action_space_type': 'discrete' # 'discrete' 'continuous'
    }



rllib_config = {
    
    'seed': 0,
    
    'env': 'ramp_job_placement_shaping_environment',
    
    'env_config': env_config,
    
    'batch_mode': 'complete_episodes',
    'train_batch_size': 4, # 1 32 128
    'sgd_minibatch_size': 4, # 1 32 128
    
    'model':{
            'fcnet_hiddens':[8],
            'fcnet_activation':'relu',
            'custom_model':'my_model',
            'custom_model_config': model_config
        },
    
    'num_workers': 4,
    'num_gpus': 1,
    
    'framework': 'torch'
    
    }

print(rllib_config)

{'seed': 0, 'env': 'ramp_job_placement_shaping_environment', 'env_config': {'node_config': {'type_1': {'num_nodes': 16, 'workers_config': [{'num_workers': 1, 'worker': <class 'ddls.devices.processors.gpus.A100.A100'>}]}}, 'topology_config': {'type': 'ramp', 'kwargs': {'num_communication_groups': 2, 'num_racks_per_communication_group': 2, 'num_servers_per_rack': 4, 'num_channels': 2}}, 'jobs_config': {'path_to_files': '/scratch/datasets/ddls/jobs/pipedream_graphs/image_classification/profiles/alexnet/', 'job_interarrival_time_dist': <ddls.distributions.uniform.Uniform object at 0x7efb574474f0>, 'max_files': 20, 'job_sampling_mode': 'remove'}, 'max_simulation_run_time': inf, 'job_queue_capacity': 100, 'reward_function': 'lookahead_job_completion_time', 'pad_obs_kwargs': {'max_nodes': 200}}, 'batch_mode': 'complete_episodes', 'train_batch_size': 4, 'sgd_minibatch_size': 4, 'model': {'fcnet_hiddens': [8], 'fcnet_activation': 'relu', 'custom_model': 'my_model', 'custom_model_config': {'in_f

In [6]:
%autoreload

# load default PPO config and update with custom config params
ppo_config = ppo.DEFAULT_CONFIG.copy()
ppo_config.update(rllib_config)
print(f'Config:\n{ppo_config}')

# initialise rllib trainer
epoch_loop = ppo.PPOTrainer(config=ppo_config)
print('\nInitialised trainer.')

2022-06-23 12:56:26,510	WARNING ppo.py:143 -- `train_batch_size` (4) cannot be achieved with your other settings (num_workers=4 num_envs_per_worker=1 rollout_fragment_length=200)! Auto-adjusting `rollout_fragment_length` to 1.
2022-06-23 12:56:26,511	INFO ppo.py:166 -- In multi-agent mode, policies will be optimized sequentially by the multi-GPU optimizer. Consider setting simple_optimizer=True if this doesn't work for you.
2022-06-23 12:56:26,512	INFO trainer.py:743 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.


Config:
{'num_workers': 4, 'num_envs_per_worker': 1, 'create_env_on_driver': False, 'rollout_fragment_length': 200, 'batch_mode': 'complete_episodes', 'gamma': 0.99, 'lr': 5e-05, 'train_batch_size': 4, 'model': {'fcnet_hiddens': [8], 'fcnet_activation': 'relu', 'custom_model': 'my_model', 'custom_model_config': {'in_features_node': 5, 'in_features_edge': 1, 'out_features_msg': 8, 'out_features_hidden': 16, 'out_features': 4, 'in_features_graph': 34, 'out_features_graph': 4, 'num_layers': 1, 'aggregator_type': 'mean', 'action_space_type': 'discrete'}}, 'optimizer': {}, 'horizon': None, 'soft_horizon': False, 'no_done_at_end': False, 'env': 'ramp_job_placement_shaping_environment', 'observation_space': None, 'action_space': None, 'env_config': {'node_config': {'type_1': {'num_nodes': 16, 'workers_config': [{'num_workers': 1, 'worker': <class 'ddls.devices.processors.gpus.A100.A100'>}]}}, 'topology_config': {'type': 'ramp', 'kwargs': {'num_communication_groups': 2, 'num_racks_per_communic

(RolloutWorker pid=2905511) 2022-06-23 12:56:34,807	WARNING catalog.py:544 -- Custom ModelV2 should accept all custom options as **kwargs, instead of expecting them in config['custom_model_config']!
(RolloutWorker pid=2905510) 2022-06-23 12:56:34,881	WARNING catalog.py:544 -- Custom ModelV2 should accept all custom options as **kwargs, instead of expecting them in config['custom_model_config']!
(RolloutWorker pid=2905509) 2022-06-23 12:56:34,865	WARNING catalog.py:544 -- Custom ModelV2 should accept all custom options as **kwargs, instead of expecting them in config['custom_model_config']!
(RolloutWorker pid=2905512) 2022-06-23 12:56:35,003	WARNING catalog.py:544 -- Custom ModelV2 should accept all custom options as **kwargs, instead of expecting them in config['custom_model_config']!
2022-06-23 12:56:36,382	WARNING catalog.py:544 -- Custom ModelV2 should accept all custom options as **kwargs, instead of expecting them in config['custom_model_config']!
2022-06-23 12:56:41,770	INFO trai


Initialised trainer.


In [ ]:
%autoreload

agent_name = 'PPO'
num_epochs = 500
rl_training_stats = defaultdict(lambda: [])
for epoch in range(num_epochs):
    print(f'\n------- Epoch {epoch+1} of {num_epochs} -------')
    result = epoch_loop.train()
    
    # print epoch data
    # print(pretty_print(result))
    
    # save epoch data
    for key, val in result['hist_stats'].items():
        rl_training_stats[key].extend(val)
    for _ in range(len(val)):
        rl_training_stats['seed'].append(result['config']['seed'])
        rl_training_stats['agent'].append(agent_name)
        rl_training_stats['epoch'].append(epoch)
        
# display(pd.DataFrame(rl_training_stats))


------- Epoch 1 of 500 -------


(RolloutWorker pid=2905509) /scratch/zciccwf/py36/envs/ddls/lib/python3.9/site-packages/numpy/core/_methods.py:179: RuntimeWarning: overflow encountered in reduce
(RolloutWorker pid=2905509)   ret = umr_sum(arr, axis, dtype, out, keepdims, where=where)
/scratch/zciccwf/py36/envs/ddls/lib/python3.9/site-packages/ray/rllib/utils/metrics/learner_info.py:84: RuntimeWarning: Mean of empty slice
  return np.nanmean(tower_data)
/scratch/zciccwf/py36/envs/ddls/lib/python3.9/site-packages/ray/rllib/utils/metrics/learner_info.py:64: RuntimeWarning: Mean of empty slice
  lambda *s: None if s[0] is None else np.nanmean(s, axis=0),


agent_timesteps_total: 4
custom_metrics: {}
date: 2022-06-23_12-56-43
done: false
episode_len_mean: 1.0
episode_media: {}
episode_reward_max: 0.0013693919704467286
episode_reward_mean: 0.0013693919650025483
episode_reward_min: 0.001369391959558368
episodes_this_iter: 4
episodes_total: 4
experiment_id: 07575fbcc2df4014a1a7692320f0bfdf
hostname: mammoth.ee.ucl.ac.uk
info:
  learner:
    default_policy:
      learner_stats:
        cur_kl_coeff: 0.20000000000000007
        cur_lr: 4.999999999999999e-05
        entropy: 2.639056841532389
        entropy_coeff: 0.0
        kl: -1.4651812515846056e-07
        policy_loss: 0.0
        total_loss: 1.1700356541410884e-07
        vf_explained_var: .nan
        vf_loss: 1.4630718965143638e-07
  num_agent_steps_sampled: 4
  num_agent_steps_trained: 4
  num_steps_sampled: 4
  num_steps_trained: 4
iterations_since_restore: 1
node_ip: 128.40.41.23
num_healthy_workers: 4
off_policy_estimator: {}
perf:
  cpu_util_percent: 24.624999999999996
  ram_util_

(RolloutWorker pid=2905512) /scratch/zciccwf/py36/envs/ddls/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
(RolloutWorker pid=2905512)   return _methods._mean(a, axis=axis, dtype=dtype,
(RolloutWorker pid=2905512) /scratch/zciccwf/py36/envs/ddls/lib/python3.9/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
(RolloutWorker pid=2905512)   ret = ret.dtype.type(ret / rcount)


agent_timesteps_total: 12
custom_metrics: {}
date: 2022-06-23_12-56-47
done: false
episode_len_mean: 1.0
episode_media: {}
episode_reward_max: 0.0013693919704467286
episode_reward_mean: -0.08208194611832222
episode_reward_min: -1.0
episodes_this_iter: 4
episodes_total: 12
experiment_id: 07575fbcc2df4014a1a7692320f0bfdf
hostname: mammoth.ee.ucl.ac.uk
info:
  learner:
    default_policy:
      learner_stats:
        cur_kl_coeff: 0.05000000000000002
        cur_lr: 4.999999999999999e-05
        entropy: 2.639053948720296
        entropy_coeff: 0.0
        kl: 1.1332590171756843e-06
        policy_loss: -0.0014623085657755534
        total_loss: 0.24834418892860413
        vf_explained_var: 2.7815500895182292e-08
        vf_loss: 0.24980644782384237
  num_agent_steps_sampled: 12
  num_agent_steps_trained: 12
  num_steps_sampled: 12
  num_steps_trained: 12
  num_steps_trained_this_iter: 0
iterations_since_restore: 3
node_ip: 128.40.41.23
num_healthy_workers: 4
off_policy_estimator: {}
perf

...

agent_timesteps_total: 548
custom_metrics: {}
date: 2022-06-23_13-01-37
done: false
episode_len_mean: 1.0
episode_media: {}
episode_reward_max: 0.0013693919704467286
episode_reward_mean: -0.038686849329026426
episode_reward_min: -1.0
episodes_this_iter: 4
episodes_total: 548
experiment_id: 07575fbcc2df4014a1a7692320f0bfdf
hostname: mammoth.ee.ucl.ac.uk
info:
  learner:
    default_policy:
      learner_stats:
        cur_kl_coeff: 2.295887403949781e-42
        cur_lr: 4.999999999999999e-05
        entropy: 2.5335166613260904
        entropy_coeff: 0.0
        kl: 0.0005620851514355726
        policy_loss: -0.058222938080628714
        total_loss: 0.17377656896909077
        vf_explained_var: 4.76837158203125e-08
        vf_loss: 0.2319995159904162
  num_agent_steps_sampled: 548
  num_agent_steps_trained: 548
  num_steps_sampled: 548
  num_steps_trained: 548
  num_steps_trained_this_iter: 0
iterations_since_restore: 137
node_ip: 128.40.41.23
num_healthy_workers: 4
off_policy_estimator: 

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/scratch/zciccwf/py36/envs/ddls/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 3457, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-7-b2c08f591d47>", line 8, in <module>
    result = epoch_loop.train()
  File "/scratch/zciccwf/py36/envs/ddls/lib/python3.9/site-packages/ray/tune/trainable.py", line 314, in train
    result = self.step()
  File "/scratch/zciccwf/py36/envs/ddls/lib/python3.9/site-packages/ray/rllib/agents/trainer.py", line 867, in step
    result = self.step_attempt()
  File "/scratch/zciccwf/py36/envs/ddls/lib/python3.9/site-packages/ray/rllib/agents/trainer.py", line 920, in step_attempt
    step_results = next(self.train_exec_impl)
  File "/scratch/zciccwf/py36/envs/ddls/lib/python3.9/site-packages/ray/util/iter.py", line 756, in __next__
    return next(self.built_iterator)
  File "/scratch/zciccwf/py36/envs/ddls/lib/python3.9/site-packages/ray/util/iter.py", line

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/scratch/zciccwf/py36/envs/ddls/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 3457, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-7-b2c08f591d47>", line 8, in <module>
    result = epoch_loop.train()
  File "/scratch/zciccwf/py36/envs/ddls/lib/python3.9/site-packages/ray/tune/trainable.py", line 314, in train
    result = self.step()
  File "/scratch/zciccwf/py36/envs/ddls/lib/python3.9/site-packages/ray/rllib/agents/trainer.py", line 867, in step
    result = self.step_attempt()
  File "/scratch/zciccwf/py36/envs/ddls/lib/python3.9/site-packages/ray/rllib/agents/trainer.py", line 920, in step_attempt
    step_results = next(self.train_exec_impl)
  File "/scratch/zciccwf/py36/envs/ddls/lib/python3.9/site-packages/ray/util/iter.py", line 756, in __next__
    return next(self.built_iterator)
  File "/scratch/zciccwf/py36/envs/ddls/lib/python3.9/site-packages/ray/util/iter.py", line

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

x = 'epoch'
scaling_factor = 1
metrics_to_plot = {'episode_reward', 'episode_lengths'}

for metric in metrics_to_plot:
    print(f'Plotting metric {metric}')
    fig = plt.figure()
    fig = plot_line(pd.DataFrame(rl_training_stats), 
                    x=x, 
                    y=metric, 
                    hue='agent', 
                    xlabel=x, 
                    ylabel=metric, 
                    err_style='band', # 'band' 'bars'
                    ci=68, # 95 68
                    scaling_factor=scaling_factor,
                    show_fig=False)
#     plt.axhline(y=np.mean(random_baseline_stats[metric]), linestyle='--', color='#a84a32', label='Random')
    plt.legend(loc='upper center', bbox_to_anchor=(0.5, 1.3), ncol=2)
    plt.show()

Plotting metric episode_lengths
ERROR! Session/line number was not unique in database. History logging moved to new session 2944


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/scratch/zciccwf/py36/envs/ddls/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 3457, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-7-a9472bd64616>", line 11, in <module>
    fig = plot_line(pd.DataFrame(rl_training_stats),
  File "/home/zciccwf/phd_project/projects/ddls/ddls/plotting/plotting.py", line 314, in plot_line
    g = sns.lineplot(data=df,
  File "/scratch/zciccwf/py36/envs/ddls/lib/python3.9/site-packages/seaborn/_decorators.py", line 46, in inner_f
    return f(**kwargs)
  File "/scratch/zciccwf/py36/envs/ddls/lib/python3.9/site-packages/seaborn/relational.py", line 710, in lineplot
    p.plot(ax, kwargs)
  File "/scratch/zciccwf/py36/envs/ddls/lib/python3.9/site-packages/seaborn/relational.py", line 499, in plot
    x, y, y_ci = self.aggregate(y, x, u)
  File "/scratch/zciccwf/py36/envs/ddls/lib/python3.9/site-packages/seaborn/relational.py", line 414, in aggregate
  

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/scratch/zciccwf/py36/envs/ddls/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 3457, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-7-a9472bd64616>", line 11, in <module>
    fig = plot_line(pd.DataFrame(rl_training_stats),
  File "/home/zciccwf/phd_project/projects/ddls/ddls/plotting/plotting.py", line 314, in plot_line
    g = sns.lineplot(data=df,
  File "/scratch/zciccwf/py36/envs/ddls/lib/python3.9/site-packages/seaborn/_decorators.py", line 46, in inner_f
    return f(**kwargs)
  File "/scratch/zciccwf/py36/envs/ddls/lib/python3.9/site-packages/seaborn/relational.py", line 710, in lineplot
    p.plot(ax, kwargs)
  File "/scratch/zciccwf/py36/envs/ddls/lib/python3.9/site-packages/seaborn/relational.py", line 499, in plot
    x, y, y_ci = self.aggregate(y, x, u)
  File "/scratch/zciccwf/py36/envs/ddls/lib/python3.9/site-packages/seaborn/relational.py", line 414, in aggregate
  

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/scratch/zciccwf/py36/envs/ddls/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 3457, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-7-a9472bd64616>", line 11, in <module>
    fig = plot_line(pd.DataFrame(rl_training_stats),
  File "/home/zciccwf/phd_project/projects/ddls/ddls/plotting/plotting.py", line 314, in plot_line
    g = sns.lineplot(data=df,
  File "/scratch/zciccwf/py36/envs/ddls/lib/python3.9/site-packages/seaborn/_decorators.py", line 46, in inner_f
    return f(**kwargs)
  File "/scratch/zciccwf/py36/envs/ddls/lib/python3.9/site-packages/seaborn/relational.py", line 710, in lineplot
    p.plot(ax, kwargs)
  File "/scratch/zciccwf/py36/envs/ddls/lib/python3.9/site-packages/seaborn/relational.py", line 499, in plot
    x, y, y_ci = self.aggregate(y, x, u)
  File "/scratch/zciccwf/py36/envs/ddls/lib/python3.9/site-packages/seaborn/relational.py", line 414, in aggregate
  